In [27]:
# %pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

# %pip install --upgrade --quiet  langchain langchain-community
# ! pip install langchainhub
# ! pip install langchain-openai
# ! pip install langchain-chroma
# ! pip install bs4
# ! pip install jq

In [76]:
import re
import sys
import traceback
# import openai
from collections import deque
from openai import OpenAI

import configparser

# import networkx as nx
import logging
import time

import os
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
from pyvis.network import Network
 

import LLM_Find_Constants as constants
from langchain_core.prompts import ChatPromptTemplate

variable_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Census_variables.csv'

In [77]:
config = configparser.ConfigParser()
config.read('config.ini')

# use your KEY.
OpenAI_key = config.get('API_Key', 'OpenAI_key')

OPENAI_API_KEY = OpenAI_key

LangChain_API_key = config.get('API_Key', 'LangChain_API_key')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY



In [78]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=OpenAI_key, model="gpt-4o")
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001889E36D070>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001889E36CA40>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')

In [59]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
# from langchain.retrievers import SimpleRetriever
# from langchain.retrievers import SimpleRetriever


from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
# loader = WebBaseLoader(
#     web_paths=("https://api.census.gov/data/2021/acs/acs5/variables.html",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer()
#     ),
# )

loader = CSVLoader(file_path=variable_fname)
# data = loader.load()
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter()  # chunk_size=10000, chunk_overlap=2000
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever( search_kwargs={"k": 150})
# retriever = SimpleRetriever.from_data(data)
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [79]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
# from langchain.retrievers import SimpleRetriever
# from langchain.retrievers import SimpleRetriever


from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
# loader = WebBaseLoader(
#     web_paths=("https://api.census.gov/data/2021/acs/acs5/variables.html",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer()
#     ),
# )

loader = CSVLoader(file_path=variable_fname)
# data = loader.load()
docs = loader.load()

vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever( search_kwargs={"k": 150})
# retriever = SimpleRetriever.from_data(data)
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [80]:
len(splits)
len(docs)

27039

In [81]:
# splits

prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [82]:
# format_docs(docs)
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000188D119DB20>, search_kwargs={'k': 150})

In [83]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [73]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001890759B170>, search_kwargs={'k': 150})

In [85]:
results = rag_chain.invoke("You need to download the senior population (older than 65) data from US Census Bureau, what variables are needed? List all possible estimation variables. Directly reply the variable names without any explanation.")
print(results)

B07001_014E, B07001_015E, B07001_016E


In [86]:
results = rag_chain.invoke("You need to obtain the  population (older than 25) by higher education attainment data from US Census Bureau, what variables are needed? List all possible estimation variables, do not select variables ending with `M` or `MA`. Directly reply the variable names without any explanation.")
print(results)

B15003_001E, B15001_015E, B15001_011E, B15001_018E, C15002F_001E, B23006_001E, B23006_026E, B15001_039E, B23006_005E, B15001_017E, B15001_014E, B29002_001E, B15001_080E, C15002F_006E, B15001_035E, C15002F_002E, B15001_025E, B23006_012E, B15001_056E, B15011_019E, B15001_012E, C15002A_001E, B15001_016E, B15001_059E, B15001_058E, B15001_052E


In [66]:
results = rag_chain.invoke("You need to download the higher education attainment population data, what variables is needed? List all possible estimation variables, do not select variables end with `M` or `MA`.")
print(results)

The possible estimation variables needed are C15002F_004E, C15002G_004E, B23006_012E, B15003_001E, B23006_010E, B15002_011E, C15002A_004E, B23006_015E, C15002F_009E, B23006_013E, B23006_009E, C15002H_004E, C15002G_003E, B23006_011E, B23006_026E, C15002F_006E, B15003_023E, C15002G_009E, B15002_007E, B15002_010E, B23006_005E, C15002B_004E, B15002_016E, C15002G_006E, C15002F_011E, B15002_008E, and C15002F_002E.


In [67]:
docs[0].page_content

'variable: B01001_001E\nlabel: Total:\nconcept: SEX BY AGE'

In [68]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", api_key=OpenAI_key)
llm

 ········


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000188DDF7E690>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000189384083E0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')

In [69]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import JSONLoader


from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.

json_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Census_variables.json"
loader = JSONLoader(
    file_path=json_fname,
    jq_schema='.messages[].content',
    text_content=False)

docs = loader.load()

docs

ImportError: jq package not found, please install it with `pip install jq`

In [ ]:


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

# Tools

In [ ]:
import json
import pandas as pd
import requests

In [ ]:
variables_url = r'https://api.census.gov/data/2019/acs/acs5/variables.json'
json_dict = requests.get(variables_url).json()

In [ ]:
json_dict['variables']['B01001_001E']

In [ ]:
# ! pip install natsort
from natsort import natsorted

In [ ]:
variable_list = natsorted(json_dict['variables'].keys())
print("Variable count:", len(variable_list))
for idx, v in enumerate(variable_list[4:]):
    print(idx, v, json_dict['variables'][v])

    if idx > 4:
        break

In [ ]:
variable_df = pd.DataFrame(json_dict['variables']).T.sort_index().iloc[4:].query("group != 'N/A'")[:-1].drop(columns=['attributes', 'required', 'hasGeoCollectionSupport', 'predicateOnly', 'limit', 'predicateType'])
variable_df.index.name = 'variable'
variable_df = variable_df.reset_index()
variable_df

In [ ]:
variable_df['top_group'] = variable_df['group'].str[:3]
variable_df['sub_group'] = variable_df['group'].str[:6]
variable_df

In [ ]:
# variable_df.to_csv(r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Census_variables.csv', index=False)

In [ ]:
variable_df['top_group'].nunique(), variable_df['sub_group'].nunique(), variable_df['concept'].nunique()

In [ ]:
for key, df in variable_df.groupby('top_group'):
    print(key, df.iloc[0]['group'], df.iloc[0]['concept'])

In [ ]:

%autoreload 2
%load_ext autoreload
    
code = helper.execute_complete_program(code=code, try_cnt=10, task=task, model_name=model_name, handbook_str=handbook_str)
display(Code(code, language='python'))
